In [3]:
import os

In [4]:
%pwd

'c:\\Users\\Parag Jadhav\\Downloads\\ML_Deployment\\red_wine_qualit_prediction\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\Parag Jadhav\\Downloads\\ML_Deployment\\red_wine_qualit_prediction'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [8]:
from src.mlproject.constant import *
from src.mlproject.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
#        params = self.params.ElasticNet
        params = self.params.RandomForestRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    
    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]
        
        y_predict = model.predict(X_test)

        (rmse, mae, r2) = self.eval_metrics(y_test, y_predict)
        
        # Saving metrics as local
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [13]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-05-11 10:54:26,798: line_number-43: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-11 10:54:26,801: line_number-43: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-11 10:54:26,808: line_number-43: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-11 10:54:26,811: line_number-63: INFO: common: created directory at: artifacts]
[2024-05-11 10:54:26,812: line_number-63: INFO: common: created directory at: artifacts/model_evaluation]
[2024-05-11 10:54:27,200: line_number-77: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
